In [1]:
%%capture
!pip install langchain langchain_core langchain_community faiss-cpu openai==1.56.2 langchain_openai langchain-huggingface

In [2]:
import os
from google.colab import userdata

# Set environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAIKey')


### Importing Libraries:

In [3]:
# LangChain imports
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from pprint import pprint

Following is a configuration function that we create in order to load our 10-K fillings:

In [39]:
defaultConfig = {
    # Document processing settings
    "chunkSize": 500,
    "chunkOverlap": 50,
    "userAgentHeader": "YourCompany-ResearchBot/1.0 (your@email.com)",

    # Embedding model
    "embeddingModel": "BAAI/bge-base-en-v1.5",

    # Vector store settings
    "numRetrievedDocuments": 5,

    # Document formatter settings
    "numSelectedDocuments": 5,

    # Model settings
    "ragAnswerModel": "gpt-4o",
    "ragAnswerModelTemeprature": 0.7,

    # URLs to process — now includes both Tesla and GM
    "companyFilingUrls": [
        ("Tesla", "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm"),
        ("GM", "https://www.sec.gov/Archives/edgar/data/1467858/000146785824000031/gm-20231231.htm")
    ],

    # RAG prompt template — focus on comparison
    "ragPromptTemplate": """
    Use only the information in the `Context` below to answer the `Question`. Focus on comparing Tesla and GM.

    Question: {question}

    Context: {context}

    Answer:
    """
}


In [41]:
config = defaultConfig.copy()

all_docs = []

# Load and label documents from both Tesla and GM
for company, url in config["companyFilingUrls"]:
    loader = WebBaseLoader(
        url,
        header_template={"User-Agent": config["userAgentHeader"]}
    )
    docs = loader.load()
    for doc in docs:
        doc.metadata["company"] = company  # Add company label
    all_docs.extend(docs)

print(f"Loaded {len(all_docs)} documents total")


Loaded 2 documents total


Splitting our data into chunks:

In [42]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config["chunkSize"],
    chunk_overlap=config["chunkOverlap"]
)
split_docs = text_splitter.split_documents(all_docs)
print(f"Split into {len(split_docs)} chunks")


Split into 2027 chunks


In [44]:
type(split_docs), len(split_docs)

(list, 2027)

In [12]:
embeddingFunction = HuggingFaceEmbeddings(
    model_name= config.get('embeddingModel','BAAI/bge-base-en-v1.5' ),
    model_kwargs={"device": "cuda"},
    encode_kwargs={
        "normalize_embeddings": True,
        "query_instruction": "Represent this question for searching relevant passages: "
    }
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [46]:
vector_store = FAISS.from_documents(split_docs, embeddingFunction)
retriever = vector_store.as_retriever(search_kwargs={"k": config["numRetrievedDocuments"]})

In [47]:
#  Basic information about the vector store
print(f"Vector store type: {type(vector_store)}")
print(f"Number of vectors: {vector_store.index.ntotal}")
print(f"Vector dimension: {vector_store.index.d}")

Vector store type: <class 'langchain_community.vectorstores.faiss.FAISS'>
Number of vectors: 2027
Vector dimension: 768


In [48]:
# Extract a sample vector to see what it looks like
sample_vector = vector_store.index.reconstruct(0)  # Get the first vector
print("\nSample vector (first 10 dimensions):")
print(sample_vector[:10])
print(f"Vector shape: {sample_vector.shape}")


Sample vector (first 10 dimensions):
[ 0.02123105  0.00325592 -0.01103543 -0.00581271  0.04987749 -0.01403621
  0.05441403  0.02510558  0.00494242 -0.01364134]
Vector shape: (768,)


In [49]:
# 1. Save locally
vector_store.save_local("/content/faiss_index")

# 2. Load later in the same session
loaded_vectorstore = FAISS.load_local(
    "/content/faiss_index",
    embeddingFunction,
    allow_dangerous_deserialization=True
)

# Verify it works
print(f"Number of vectors loaded: {loaded_vectorstore.index.ntotal}")

Number of vectors loaded: 2027


In [50]:
# First ensure you've loaded your vector store
loaded_vectorstore = FAISS.load_local(
    "/content/faiss_index",  # Your local path
    embeddingFunction,
    allow_dangerous_deserialization=True
)

# Get the mapping dictionary
index_dict = loaded_vectorstore.index_to_docstore_id

# Display the first 10 mappings
print(f"\nFirst {min(10, len(index_dict))} items in index_to_docstore_id:")
print("-" * 60)
for i, (faiss_id, doc_id) in enumerate(index_dict.items()):
    if i >= 10:  # Only show first 10
        break
    print(f"FAISS ID: {faiss_id:>3} → Document ID: {doc_id}")


First 10 items in index_to_docstore_id:
------------------------------------------------------------
FAISS ID:   0 → Document ID: c169c892-05de-4df8-bfa2-a7410cc910fb
FAISS ID:   1 → Document ID: 1d310c9b-42be-4561-993f-2c85e9808d65
FAISS ID:   2 → Document ID: 425552c3-74a7-4210-8d83-2cc2abfc4cb3
FAISS ID:   3 → Document ID: 747b8353-6cc5-4975-a83f-6b5fc893a591
FAISS ID:   4 → Document ID: 92152d32-1789-4b14-9d9e-bff6f18651d3
FAISS ID:   5 → Document ID: f87ca66d-80a8-467a-beaf-d6dd7a807284
FAISS ID:   6 → Document ID: de00b00b-030b-47fa-9fb0-c32431332619
FAISS ID:   7 → Document ID: 506a4984-557f-4a42-ad77-cf775f30edda
FAISS ID:   8 → Document ID: 1063d27d-fed5-4347-9c7e-55e0c7f76f5e
FAISS ID:   9 → Document ID: 949fb7fa-eea3-4690-84ab-0628852db6c8


In [52]:
# Get information about a specific chunk (the 100th vector)
# Convert to string since keys are often stored as strings
faiss_id = "100" if "100" in index_dict else 100

# Get the document ID corresponding to the 100th vector
document_id = loaded_vectorstore.index_to_docstore_id[faiss_id]
print(f"\nFAISS ID: {faiss_id}")
print(f"Document ID: {document_id}")

# Get document
document = loaded_vectorstore.docstore._dict[document_id]
print("\nDOCUMENT METADATA:")
pprint(document.metadata)
print(f"Company: {document.metadata.get('company', 'N/A')}")
print("\nDOCUMENT CONTENT:")
pprint(document.page_content)

# Get dopcument vector
vector = loaded_vectorstore.index.reconstruct(int(faiss_id))
print(f"\nVector dimensions: {len(vector)}")
print(f"\nFirst 10 values: {vector[:10]}")



FAISS ID: 100
Document ID: ef2d66c8-351c-462c-a7bb-55ccecc1b55c

DOCUMENT METADATA:
{'company': 'Tesla',
 'language': 'No language found.',
 'source': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm',
 'title': 'tsla-20231231'}
Company: Tesla

DOCUMENT CONTENT:
('dangerous aspects of road travel much like the system that airplane pilots '
 'use, when conditions permit. As with other vehicle systems, we improve these '
 'functions in our vehicles over time through over-the-air updates.We intend '
 'to establish in the future an autonomous Tesla ride-hailing network, which '
 'we expect would also allow us to access a new customer base even as modes of '
 'transportation evolve.We are also applying our artificial intelligence '
 'learnings from self-driving technology to the')

Vector dimensions: 768

First 10 values: [ 0.02108986  0.01807349  0.04688764  0.02431504  0.04146757 -0.0005477
  0.04224639  0.01236367  0.02040216 -0.04046223]


In [54]:
# Perform similarity search
query = "What technological advancements were made in the batteries used in GM's Electric Vehicles?"
print(f"\n=== Similarity Search Results for Query ===\n{query}")

# Perform the search with scores
search_results = loaded_vectorstore.similarity_search_with_score(query, k=5)

# Display results with vectors
for i, (doc, score) in enumerate(search_results):
    print(f"\nResult {i+1} (Similarity Score: {score}):")
    pprint(f"Content: {doc.page_content}")
    pprint(f"Company: {doc.metadata.get('company', 'Unknown')}")
    pprint(f"Metadata: {doc.metadata}")

    # Find the document ID that matches this result
    result_doc_id = None
    for id, stored_doc in loaded_vectorstore.docstore._dict.items():
        if stored_doc.page_content == doc.page_content:
            result_doc_id = id
            break

    if result_doc_id:
        # Find the corresponding vector
        if hasattr(loaded_vectorstore, "index_to_docstore_id"):
            for faiss_id, doc_id in loaded_vectorstore.index_to_docstore_id.items():
                if doc_id == result_doc_id:
                    vector = loaded_vectorstore.index.reconstruct(int(faiss_id))
                    print(f"Vector dimensions: {len(vector)}")
                    print(f"Vector (first 5 dimensions): {vector[:5]}")
                    break

print("\nSearch complete!")


=== Similarity Search Results for Query ===
What technological advancements were made in the batteries used in GM's Electric Vehicles?

Result 1 (Similarity Score: 0.5443627834320068):
('Content: technology featuring three electric motors for further increased '
 'performance in certain versions of Model S and Model X, Cybertruck and the '
 'Tesla Semi.We maintain extensive testing and R&D capabilities for battery '
 'cells, packs and systems, and have built an expansive body of knowledge on '
 'lithium-ion cell chemistry types and performance characteristics. In order '
 'to enable a greater supply of cells for our products with higher energy '
 'density at lower costs, we have developed a new proprietary')
'Company: Tesla'
("Metadata: {'source': "
 "'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm', "
 "'title': 'tsla-20231231', 'language': 'No language found.', 'company': "
 "'Tesla'}")
Vector dimensions: 768
Vector (first 5 dimensions): [ 0.021

In [55]:
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": config["numRetrievedDocuments"]})

In [56]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79fa043e5e90>, search_kwargs={'k': 5})

In [57]:
retrieved_documents = retriever.invoke("What technological advancements were made in the batteries used in Tesla's Electric Vehicles?")

In [58]:
retrieved_documents

[Document(id='d903d229-f69a-426b-a35d-d2f98bbf61c8', metadata={'source': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm', 'title': 'tsla-20231231', 'language': 'No language found.', 'company': 'Tesla'}, page_content='technology featuring three electric motors for further increased performance in certain versions of Model S and Model X, Cybertruck and the Tesla Semi.We maintain extensive testing and R&D capabilities for battery cells, packs and systems, and have built an expansive body of knowledge on lithium-ion cell chemistry types and performance characteristics. In order to enable a greater supply of cells for our products with higher energy density at lower costs, we have developed a new proprietary'),
 Document(id='bc5287b6-5a4b-499e-9b90-99b9f1ec7e01', metadata={'source': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm', 'title': 'tsla-20231231', 'language': 'No language found.', 'company': 'Tesla'}, pag

In [59]:
def format_docs(docs):
    return "\n\n".join([
        f"{doc.metadata.get('company', '')}\n{doc.page_content}"
        for doc in docs[:config["numSelectedDocuments"]]
    ])

In [60]:
context = format_docs(retrieved_documents)
pprint(context)

('Tesla\n'
 'technology featuring three electric motors for further increased performance '
 'in certain versions of Model S and Model X, Cybertruck and the Tesla Semi.We '
 'maintain extensive testing and R&D capabilities for battery cells, packs and '
 'systems, and have built an expansive body of knowledge on lithium-ion cell '
 'chemistry types and performance characteristics. In order to enable a '
 'greater supply of cells for our products with higher energy density at lower '
 'costs, we have developed a new proprietary\n'
 '\n'
 'Tesla\n'
 'public attention, especially in light of NHTSA’s Standing General Order '
 'requiring reports regarding crashes involving vehicles with advanced driver '
 'assistance systems. We have experienced, and we expect to continue to face, '
 'claims and regulatory scrutiny arising from or related to misuse or claimed '
 'failures or alleged misrepresentations of such new technologies that we are '
 'pioneering. In addition, the battery packs that w

In [68]:
passthrough = RunnablePassthrough()
question = passthrough.invoke("How do Tesla and GM's approaches to manufacturing and production compare, particularly for electric vehicles? Where are their vehicles produced? What are the safety standards followed in their vehicles?")
print(question)


How do Tesla and GM's approaches to manufacturing and production compare, particularly for electric vehicles? Where are their vehicles produced? What are the safety standards followed in their vehicles?


In [69]:
prompt_template = PromptTemplate.from_template(config["ragPromptTemplate"])
prompt_template

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    Use only the information in the `Context` below to answer the `Question`. Focus on comparing Tesla and GM.\n\n    Question: {question}\n\n    Context: {context}\n\n    Answer:\n    ')

In [70]:
# Create the formatted prompt by filling in the template
formatted_prompt = prompt_template.format(question=question, context=context)
pprint(formatted_prompt)

('\n'
 '    Use only the information in the `Context` below to answer the '
 '`Question`. Focus on comparing Tesla and GM.\n'
 '\n'
 "    Question: How do Tesla and GM's approaches to manufacturing and "
 'production compare, particularly for electric vehicles? Where are their '
 'vehicles produced? What are the safety standards followed in their '
 'vehicles?\n'
 '\n'
 '    Context: Tesla\n'
 'technology featuring three electric motors for further increased performance '
 'in certain versions of Model S and Model X, Cybertruck and the Tesla Semi.We '
 'maintain extensive testing and R&D capabilities for battery cells, packs and '
 'systems, and have built an expansive body of knowledge on lithium-ion cell '
 'chemistry types and performance characteristics. In order to enable a '
 'greater supply of cells for our products with higher energy density at lower '
 'costs, we have developed a new proprietary\n'
 '\n'
 'Tesla\n'
 'public attention, especially in light of NHTSA’s Standing Ge

# Simple RAG:

In [71]:
# Set up OpenAI LLM
llm = ChatOpenAI(
    model=config["ragAnswerModel"],
    temperature=config["ragAnswerModelTemeprature"]
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x79fa062347d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x79fa045c6550>, root_client=<openai.OpenAI object at 0x79fa0429c0d0>, root_async_client=<openai.AsyncOpenAI object at 0x79fa0435a550>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [72]:
llm_output = llm.invoke(formatted_prompt)
llm_output

AIMessage(content="Tesla and GM have different approaches to manufacturing and production, particularly for electric vehicles (EVs).\n\n**Tesla:**\n- **Manufacturing and Production:** Tesla has developed proprietary lithium-ion battery cells to achieve higher energy density at lower costs. They have extensive testing and R&D capabilities focused on battery cells, packs, and systems, and they design most of their sophisticated control and infotainment software internally.\n- **Production Locations:** Tesla's vehicles, like the Model S, Model X, Cybertruck, and Tesla Semi, are produced using advanced manufacturing processes, though the specific production sites were not detailed in the context provided.\n- **Safety Standards:** Tesla vehicles utilize advanced performance and safety systems managed by sophisticated control software. They face regulatory scrutiny and claims regarding their advanced driver assistance systems, indicating a focus on meeting safety standards amid evolving tech

In [73]:
pprint(llm_output.content)

('Tesla and GM have different approaches to manufacturing and production, '
 'particularly for electric vehicles (EVs).\n'
 '\n'
 '**Tesla:**\n'
 '- **Manufacturing and Production:** Tesla has developed proprietary '
 'lithium-ion battery cells to achieve higher energy density at lower costs. '
 'They have extensive testing and R&D capabilities focused on battery cells, '
 'packs, and systems, and they design most of their sophisticated control and '
 'infotainment software internally.\n'
 "- **Production Locations:** Tesla's vehicles, like the Model S, Model X, "
 'Cybertruck, and Tesla Semi, are produced using advanced manufacturing '
 'processes, though the specific production sites were not detailed in the '
 'context provided.\n'
 '- **Safety Standards:** Tesla vehicles utilize advanced performance and '
 'safety systems managed by sophisticated control software. They face '
 'regulatory scrutiny and claims regarding their advanced driver assistance '
 'systems, indicating a focus

In [74]:
output_parser = StrOutputParser()
final_output = output_parser.invoke(llm_output)
pprint(final_output)

('Tesla and GM have different approaches to manufacturing and production, '
 'particularly for electric vehicles (EVs).\n'
 '\n'
 '**Tesla:**\n'
 '- **Manufacturing and Production:** Tesla has developed proprietary '
 'lithium-ion battery cells to achieve higher energy density at lower costs. '
 'They have extensive testing and R&D capabilities focused on battery cells, '
 'packs, and systems, and they design most of their sophisticated control and '
 'infotainment software internally.\n'
 "- **Production Locations:** Tesla's vehicles, like the Model S, Model X, "
 'Cybertruck, and Tesla Semi, are produced using advanced manufacturing '
 'processes, though the specific production sites were not detailed in the '
 'context provided.\n'
 '- **Safety Standards:** Tesla vehicles utilize advanced performance and '
 'safety systems managed by sophisticated control software. They face '
 'regulatory scrutiny and claims regarding their advanced driver assistance '
 'systems, indicating a focus

In [75]:
# Create retriever
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": config["numRetrievedDocuments"]})

# Define formatting function
def format_docs(docs):
    return "\n\n".join([
        f"[{doc.metadata.get('company', 'Unknown')}]\n{doc.page_content}"
        for doc in docs[:config["numSelectedDocuments"]]
    ])

# Create LLM
llm = ChatOpenAI(model=config["ragAnswerModel"], temperature=config["ragAnswerModelTemeprature"])

# Create prompt
prompt = PromptTemplate.from_template(config["ragPromptTemplate"])

# Build RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [76]:
question = "How do Tesla and GM's approaches to manufacturing and production compare, particularly for electric vehicles? Where are their vehicles produced? What are the safety standards followed in their vehicles?"
answer = rag_chain.invoke(question)
print(answer)

Tesla and GM have distinct approaches to manufacturing and production, particularly regarding electric vehicles (EVs). Tesla focuses on creating a complete energy and transportation ecosystem to accelerate the transition to sustainable energy, which involves designing and building new factories to support this goal. On the other hand, GM is transitioning to an all-electric future with a focus on developing a resilient, scalable, and sustainable North America-focused EV supply chain.

In terms of production locations, GM's vehicles, particularly the BrightDrop's Zevo 600 and Zevo 400, are produced at the CAMI Assembly in Canada, which is noted as Canada's first full-scale EV manufacturing facility. GM also plans to mass-produce battery cells in the United States through its joint venture with LG Energy Solution in Warren, Ohio; Spring Hill, Tennessee; and Lansing, Michigan.

Tesla, while not explicitly detailed in the provided context, is known for its production facilities in the Unite

# Query Decomposition Approach:

In [77]:
# Decomposition prompt
decomposition_prompt = PromptTemplate.from_template("""
Break down the `Question` into multiple sub-queries. Use the guidelines given below to help in the task.

1. The set of sub-queries together capture the complete information needed to answer the question.
2. Each sub-query should ask for just one piece of information about one specific company.
3. For each sub-query, only mention the information you're trying to get. Don't use verbs like "retrieve" or "find".
4. Include the company name mentioned in each sub-query.
5. Do not include any references to data sources in your sub-queries.

Enclose the sub-query in angle brackets. For example:

<sub-query 1>
<sub-query 2>

Question: {question}

Begin:
""")

# Chain to run the prompt
decomposition_chain = decomposition_prompt | llm | StrOutputParser()


In [78]:
# Define the main question
main_question = (
    "How do Tesla and GM's approaches to manufacturing and production compare, "
    "particularly for electric vehicles? Where are their vehicles produced? "
    "What are the safety standards followed in their vehicles?"
)

# Run decomposition
decomposed_output = decomposition_chain.invoke({"question": main_question})
print("=== Decomposed Sub-questions ===")
print(decomposed_output)


=== Decomposed Sub-questions ===
<Manufacturing approach of Tesla>  
<Manufacturing approach of GM>  
<Production approach of Tesla for electric vehicles>  
<Production approach of GM for electric vehicles>  
<Production locations of Tesla vehicles>  
<Production locations of GM vehicles>  
<Safety standards followed in Tesla vehicles>  
<Safety standards followed in GM vehicles>  


In [79]:
import re

# Extract sub-queries from brackets
sub_questions = re.findall(r"<(.*?)>", decomposed_output)
print("\nParsed sub-questions:")
for sq in sub_questions:
    print(f"• {sq}")



Parsed sub-questions:
• Manufacturing approach of Tesla
• Manufacturing approach of GM
• Production approach of Tesla for electric vehicles
• Production approach of GM for electric vehicles
• Production locations of Tesla vehicles
• Production locations of GM vehicles
• Safety standards followed in Tesla vehicles
• Safety standards followed in GM vehicles


In [80]:
answers = []
for i, sub_q in enumerate(sub_questions):
    print(f"\n=== Sub-question {i+1} ===\n{sub_q}")
    answer = rag_chain.invoke(sub_q)
    print(f"\n🟩 Answer:\n{answer}\n")
    answers.append((sub_q, answer))



=== Sub-question 1 ===
Manufacturing approach of Tesla

🟩 Answer:
Tesla's manufacturing approach is characterized by several innovative strategies. They focus on reducing product development time through crash test simulations and have achieved significant advancements in engineering related to stamping, casting, and thermal systems. One notable innovation is their method of integrating batteries directly into vehicle body structures, optimizing manufacturability, weight, range, and cost. Tesla is also expanding its global manufacturing operations with a focus on localizing vehicle designs and production. Their approach includes developing their own battery cells to improve vehicle performance, decrease production costs, and increase affordability. Additionally, Tesla's Manufacturing Development Program aims to prepare high school graduates for careers in manufacturing at Tesla, with plans to expand this initiative.


=== Sub-question 2 ===
Manufacturing approach of GM

🟩 Answer:
GM e

In [81]:
synthesis_prompt = PromptTemplate.from_template("""
You are an expert in analyzing corporate manufacturing strategies.

Using the answers to the sub-questions below, synthesize a comprehensive, comparative response to the original question.

Make sure your response:
- Clearly compares Tesla and GM's approaches
- Separates their manufacturing strategies, locations, and safety standards
- Avoids repeating the sub-answers word-for-word
- Presents the information in a natural, coherent, and professional tone

Original Question:
{question}

Sub-Answers:
{sub_answers}

Final Answer:
""")


In [82]:
# Build string of sub-answers
formatted_sub_answers = "\n".join(
    [f"Q: {q}\nA: {a}" for q, a in answers]
)


In [83]:
synthesis_chain = synthesis_prompt | llm | StrOutputParser()

final_response = synthesis_chain.invoke({
    "question": main_question,
    "sub_answers": formatted_sub_answers
})

print("=== FINAL SYNTHESIZED ANSWER ===\n")
print(final_response)


=== FINAL SYNTHESIZED ANSWER ===

When comparing the manufacturing and production strategies of Tesla and General Motors (GM), particularly in the realm of electric vehicles (EVs), both companies exhibit distinctive approaches that reflect their unique corporate philosophies and market strategies.

**Manufacturing Approaches:**

Tesla's strategy is heavily innovation-driven, focusing on integrating cutting-edge technology and developing proprietary processes. The company emphasizes rapid product development through simulations and has pioneered techniques such as integrating batteries directly into vehicle structures, which enhances manufacturability and performance. Tesla's commitment to innovation extends to its in-house battery cell development, aimed at enhancing vehicle performance and reducing production costs. Additionally, Tesla's Manufacturing Development Program is a workforce initiative designed to cultivate talent directly from high school graduates, indicating a long-term 